In [2]:
import torch
from torch.utils.data import DataLoader
import tqdm
from datasets import load_dataset
from torchvision.models import resnet50
from torchvision import transforms
from torchvision.datasets import ImageFolder
import torch.nn as nn


/home2/esh/miniconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = load_dataset("Maysee/tiny-imagenet")

Found cached dataset parquet (/home2/esh/.cache/huggingface/datasets/Maysee___parquet/Maysee--tiny-imagenet-35af7c46a941f08e/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 2/2 [00:00<00:00, 167.94it/s]


In [4]:
train = dataset["train"]
val = dataset["valid"]

In [5]:
train

Dataset({
    features: ['image', 'label'],
    num_rows: 100000
})

In [6]:
(train[7]['image'].size)

(64, 64)

In [7]:
class TinyImageNetDataset(torch.utils.data.Dataset):

    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __getitem__(self, index):
        image = self.dataset[index]["image"]
        label = self.dataset[index]["label"]

        if self.transform:
            image = self.transform(image)
        # if grayscale, convert to 3-channel
        if image.size(0) == 1:
            image = image.repeat(3, 1, 1)
            
        label = torch.tensor(label)
        return image, label

    def __len__(self):
        return len(self.dataset)

In [8]:
transform = transforms.Compose([
    transforms.ToTensor(),
])


In [9]:
train_dataset = TinyImageNetDataset(train, transform)
val_dataset = TinyImageNetDataset(val, transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)

In [10]:
print(len(train_loader))
print(len(train_dataset))

1563
100000


In [11]:
for x, y in train_loader:
    print(x.size(), y.size())
    break

torch.Size([64, 3, 64, 64]) torch.Size([64])


In [12]:
BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 0.001
MOMENTUM = 0.9

In [19]:
class Resnet50TinyImageNet(nn.Module):
  # load the pretrained ResNet50 model
  # freeze the weights of the pre-trained layers
  # modify the last layer to output 200 classes
    def __init__(self):
        super(Resnet50TinyImageNet, self).__init__()
        self.model = resnet50()
        for param in self.model.parameters():
            param.requires_grad = False
        num_features = self.model.fc.in_features
        self.model.fc = nn.Linear(num_features, 200)
    
    def forward(self, x):
        return self.model(x)
    
    def train(self, train_loader, criterion, optimizer, num_epochs=10):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        optimizer = torch.optim.SGD(self.model.fc.parameters(), lr=0.001, momentum=0.9)

        self.model.to(device)
        
        for epoch in range(num_epochs):
            train_loss = 0.0
            
            self.model.train()
            for image, label in tqdm.tqdm(train_loader):
                image = image.to(device)
                label = label.to(device)
                
                optimizer.zero_grad()
                
                outputs = self.model(image)
                loss = criterion(outputs, label)
                loss.backward()
                optimizer.step()
                
                train_loss += loss.item() * image.size(0)
                _, prediction = torch.max(outputs, 1)
            
            train_loss = train_loss / len(train_loader.dataset)

            print('Epoch: {} \tTraining Loss: {:.6f}'.format(
                epoch+1, train_loss))
            

In [20]:
model = Resnet50TinyImageNet()

criterion = nn.CrossEntropyLoss()

model.train(train_loader, criterion, EPOCHS)

100%|██████████| 1563/1563 [01:23<00:00, 18.66it/s]


Epoch: 1 	Training Loss: 5.372446


 29%|██▉       | 459/1563 [00:24<00:58, 18.76it/s]


KeyboardInterrupt: 